In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as ad
import os
import scanpy as sc
from statsmodels.stats.multitest import multipletests
import sys
sys.path.append("../src")
from evaluation_metrics import *

In [3]:
xm_results = "../evaluation_results/1_1_monotonicity_scxmatch/"
xm_result_dfs = {f.split("_results.txt")[0]: pd.read_csv(os.path.join(xm_results, f)) for f in os.listdir(xm_results) if f.endswith("results.txt")}

In [11]:
bm_results = "../evaluation_results/1_1_monotonicity_benchmark/"
bm_result_dfs = {f.split("benchmark_results_processed_")[1].split(".")[0]: pd.read_csv(os.path.join(bm_results, f), index_col=0) for f in os.listdir(bm_results) if f.endswith("csv")}

In [12]:
mon = dict()
for dataset in bm_result_dfs:
    print(dataset)
    bm_result_dfs[dataset] = bm_result_dfs[dataset].sort_index()
    bm_result_dfs[dataset]["scXMatch"] = 1 - xm_result_dfs[dataset].sort_values("test_group")["p"].values
    bm_result_dfs[dataset][['wilcoxon', 'deseq2_100', 'edgeR_100', 'deseq2_200',
       'edgeR_200', 'deseq2_500', 'edgeR_500']] = bm_result_dfs[dataset][['wilcoxon', 'deseq2_100', 'edgeR_100', 'deseq2_200',
       'edgeR_200', 'deseq2_500', 'edgeR_500']].astype(float) / 2000
    mon[dataset] = bm_result_dfs[dataset].apply(monotonicity, axis=0)

mcfarland
6     0.725000
24    0.678934
Name: augur, dtype: float64
[0.72 0.68]
6     0.4280
24    0.3485
Name: wilcoxon, dtype: float64
[0.43 0.35]
6     0.1875
24    0.3120
Name: deseq2_100, dtype: float64
[0.19 0.31]
6     0.222
24    0.369
Name: edgeR_100, dtype: float64
[0.22 0.37]
6     0.1785
24    0.3035
Name: deseq2_200, dtype: float64
[0.18 0.3 ]
6     0.2115
24    0.3350
Name: edgeR_200, dtype: float64
[0.21 0.34]
6     0.103
24    0.192
Name: deseq2_500, dtype: float64
[0.1  0.19]
6     0.1730
24    0.2825
Name: edgeR_500, dtype: float64
[0.17 0.28]
6     1.0
24    1.0
Name: scXMatch, dtype: float64
[1. 1.]
norman
1    0.523027
2    0.636950
Name: augur, dtype: float64
[0.52 0.64]
1    0.6670
2    0.7125
Name: wilcoxon, dtype: float64
[0.67 0.71]
1    0.5845
2    0.6405
Name: deseq2_100, dtype: float64
[0.58 0.64]
1    0.6625
2    0.7120
Name: edgeR_100, dtype: float64
[0.66 0.71]
1    0.599
2    0.653
Name: deseq2_200, dtype: float64
[0.6  0.65]
1    0.6535
2    0.7045
Nam

In [14]:
pd.DataFrame(mon)

,mcfarland,norman,schiebinger,sciplex_A549,sciplex_K562,sciplex_MCF7
augur,0.0,1.0,0.734403,0.666667,0.666667,1.0
wilcoxon,0.0,1.0,0.926916,0.833333,1.000000,1.0
deseq2_100,1.0,1.0,0.925134,1.000000,1.000000,1.0
edgeR_100,1.0,1.0,0.935829,1.000000,1.000000,1.0
deseq2_200,1.0,1.0,0.953654,1.000000,1.000000,1.0
edgeR_200,1.0,1.0,0.950089,1.000000,1.000000,1.0
deseq2_500,1.0,1.0,0.962567,1.000000,1.000000,1.0
edgeR_500,1.0,1.0,0.946524,1.000000,1.000000,1.0
scXMatch,1.0,1.0,1.000000,1.000000,0.666667,1.0


In [ ]:
results_within = "../evaluation_results/within/"
result_dfs_within = {f.split("_results.txt")[0].split("within_")[1]: pd.read_csv(os.path.join(results_within, f)).set_index("test_group") for f in os.listdir(results_within) if "within" in f}

In [ ]:
result_dfs_within["norman"]
for dataset in result_dfs:
    print(dataset)
    ps = list()
    zs = list()
    for group in result_dfs_within[dataset]["reference"].unique():
        if group in ["control", "0.0", 0.0]:
            print(group)
            continue
        sub_res = result_dfs_within[dataset].loc[group]
        ps.append(sub_res["p"].values)
        zs.append(sub_res["z"].values)
    ps = np.stack(ps)
    zs = np.stack(zs)
    print(result_dfs[dataset]["p"].shape)
    print(ps.shape)
    print(dataset, SSNR(result_dfs[dataset]["p"], ps, smaller_is_stronger=True), SSNR(result_dfs[dataset]["z"], zs, smaller_is_stronger=True))
    #    p_mon = monotonicity(result_dfs[dataset].sort_values("test_group")["p"], smaller_is_stronger=True)
    #    z_mon = monotonicity(result_dfs[dataset].sort_values("test_group")["z"], smaller_is_stronger=True)
    #print(dataset, p_mon, z_mon)

In [ ]:
result_dfs["sciplex_MCF7"]

In [ ]:
alpha = 0.05
fig = plt.figure(figsize=(12, 6))
gs = fig.add_gridspec(3, 5, width_ratios=[1, 1, 2, 2, 2])

ax1 = fig.add_subplot(gs[0, 0], sharey=None)
ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
ax3 = fig.add_subplot(gs[0, 2], sharey=ax1)
ax4 = fig.add_subplot(gs[0, 3], sharey=ax1)
ax5 = fig.add_subplot(gs[0, 4], sharey=ax1)
ax6 = fig.add_subplot(gs[1, :], sharey=ax1)
ax7 = fig.add_subplot(gs[2, :], sharey=ax1)

axs = [ax1, ax2, ax3, ax4, ax5, ax6, ax7]

vmin = 0
vmax = 0.3

datasets = ["mcfarland", "norman", "sciplex_A549", "sciplex_K562", "sciplex_MCF7", "schiebinger"]
names = ["McFarland", "Norman", "Sciplex A549", "Sciplex K562", "Sciplex MCF7", "Schiebinger"]

for i, (dataset, name) in enumerate(zip(datasets, names)):
    data = result_dfs[dataset][["$P_{adj}$", "$cov$", "test_group"]].set_index("test_group").T
    if dataset == "schiebinger":
        data.columns = data.columns.astype(float)
        data = data.sort_index(axis=1)
        data1 = data.iloc[:, :len(data.columns)//2]
        data2 = data.iloc[:, len(data.columns)//2:]        
        sns.heatmap(data1 >= alpha, annot=data1, square=True, ax=axs[i], cbar=False, vmin=vmin, vmax=vmax, linewidth=.5, fmt=".3f", cmap="Set2", annot_kws={"size": 8})
        sns.heatmap(data2 >= alpha, annot=data2, square=True, ax=axs[i + 1], cbar=False, vmin=vmin, vmax=vmax, linewidth=.5, fmt=".3f", cmap="Set2", annot_kws={"size": 8})
    else:
        data.columns = data.columns.astype(int)
        sns.heatmap(data > alpha, annot=data, square=True, ax=axs[i], cbar=True, vmin=vmin, vmax=vmax, linewidth=.5, fmt=".3f", cmap="Set2", annot_kws={"size": 8})
    axs[i].set_title(name)

#plt.tight_layout()
plt.savefig("../plots/fig3/scrnaseq1.pdf", bbox_inches="tight")
plt.show()

In [ ]:
alpha = 0.05
fig = plt.figure(figsize=(12, 6))
gs = fig.add_gridspec(3, 5, width_ratios=[1, 1, 2, 2, 2])

ax1 = fig.add_subplot(gs[0, 0], sharey=None)
ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
ax3 = fig.add_subplot(gs[0, 2], sharey=ax1)
ax4 = fig.add_subplot(gs[0, 3], sharey=ax1)
ax5 = fig.add_subplot(gs[0, 4], sharey=ax1)
ax6 = fig.add_subplot(gs[1, :], sharey=ax1)
ax7 = fig.add_subplot(gs[2, :], sharey=ax1)

axs = [ax1, ax2, ax3, ax4, ax5, ax6, ax7]

vmin = -80
vmax = 0

datasets = ["mcfarland", "norman", "sciplex_A549", "sciplex_K562", "sciplex_MCF7", "schiebinger"]
names = ["McFarland", "Norman", "Sciplex A549", "Sciplex K562", "Sciplex MCF7", "Schiebinger"]

for i, (dataset, name) in enumerate(zip(datasets, names)):
    data = result_dfs[dataset][["$z$", "test_group"]].set_index("test_group").T
    if dataset == "schiebinger":
        data = data.sort_index(axis=1)
        data.columns = data.columns.astype(float)
        data1 = data.iloc[:, :len(data.columns)//2]
        data2 = data.iloc[:, len(data.columns)//2:]        
        sns.heatmap(data1, annot=data1, square=True, ax=axs[i], cbar=True, vmin=vmin, vmax=vmax, linewidth=.5, fmt=".3f", cmap="magma", annot_kws={"size": 8})
        sns.heatmap(data2, annot=data2, square=True, ax=axs[i + 1], cbar=True, vmin=vmin, vmax=vmax, linewidth=.5, fmt=".3f", cmap="magma", annot_kws={"size": 8})
    else:
        data.columns = data.columns.astype(int)
        sns.heatmap(data, annot=data, square=True, ax=axs[i], cbar=True, vmin=vmin, vmax=vmax, linewidth=.5, fmt=".3f", cmap="magma", annot_kws={"size": 8})
    axs[i].set_title(name)

plt.tight_layout()
plt.savefig("../plots/fig3/scrnaseq2.pdf", bbox_inches="tight")
plt.show()